In [2]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)


In [3]:
model.load_state_dict(torch.load('best_model.pth'))

In [4]:
device = torch.device('cuda')
model = model.to(device)

In [5]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [7]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

In [8]:
camera_link.unlink()  # don't stream to browser (will still run camera)

In [24]:
from jetbot import Robot

robot = Robot()


In [30]:
camera.unobserve(update, names='value')
robot.stop()

In [28]:
import torch.nn.functional as F
import time
moveval = 0

def update(change):
    global blocked_slider, robot, moveval
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.769)
        moveval = 0
    else:
        robot.left(0.6)
        moveval = moveval + 1    
        if moveval > 5:
            robot.backward(0.45)
            robot.left(.55)
            robot.backward(0.45)
            
            
    time.sleep(0.01)
        
update({'new': camera.value})  # we call the function once to intialize


In [29]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

182
beep...beep...beep...
